In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
#датафрейм, где в первой колонке -- целевая переменная, в остальных -- предикторы с подходящими лагами
d = {'Volume of mortgage loans': volume, 'Russian Central Bank Key rate': rate[:-3], 'Consumer price index': index[3:], 'Average apartment price': prices[3:],
     'Dollar/ruble exchange rate': usd[2:-1], 'Euro/ruble exchange rate': euro[3:], 'Crude oil price': oil[:-3], 'Yandex Query 1': yandex1[2:-1], 
     'Yandex Query 2': yandex2[2:-1]}
lagged_df = pd.DataFrame(d)

In [ ]:
lagged_df['date'] = df_volume.index
lagged_df = lagged_df.set_index('date')
size = int(len(lagged_df) * 0.7)
df_train, df_test = lagged_df[:size], lagged_df[size:]

In [ ]:
#проверим целевой ряд на стационарность тестом Дики-Фуллера. Если значение p-value < 0.05, то гипотеза о существовании 
#единичного корня отвергается, d=0
print("Dickey-Fuller test: p=%f" % sm.tsa.stattools.adfuller(df_train['Volume of mortgage loans'])[1])

In [ ]:
#если недостаточно оснований для того, чтобы отвергнуть гипотезу (p>0.05), то дифференцируем ряд (теперь будет d=1)
df_train['data_diff'] = df_train['trains'] - df_train['trains'].shift(1)
#и снова проверяем его на стационарность
print("Dickey-Fuller test: p=%f" % sm.tsa.stattools.adfuller(df_train.data_diff[1:])[1])
#возможно, придется еще раз дифференцировать или применить какое-то другое преобразование

In [ ]:
#d - порядок дифференцирования, количество лагов можно варьировать в зависимости от длины ряда
# autocorrelation
%pylab inline
plt.figure(figsize(12,6))
ax = plt.subplot(211)
sm.graphics.tsa.plot_acf(df_train['Volume of mortgage loans'][d:].values.squeeze(), lags=23, ax=ax)
pylab.show()

# partial autocorrelation 
ax = plt.subplot(212)
sm.graphics.tsa.plot_pacf(df_train['Volume of mortgage loans'][d:].values.squeeze(), lags=10, ax=ax)
pylab.show()

In [ ]:
#d уже известен. p смотрим на графике partial autocorrelations, q на графике autocorrelations (последний значимый лаг)
from itertools import product
ps = range(0, p)
d  = d
qs = range(0, q)

parameters = product(ps, qs)
parameters_list = list(parameters)
print ("Number of analysed models:", len(parameters_list))

In [ ]:
#перебором находим лучшую комбинацию параметров
%time
results = []
best_aic = float("inf")

# ignore warnings 
warnings.filterwarnings('ignore')

for param in parameters_list:
    
    #try except for the case of wrong parameters
    try:
        model=sm.tsa.statespace.SARIMAX(df_train['Volume of mortgage loans'], order=(param[0], d, param[1]), seasonal_order=(0,0,0,0)).fit()
        
    #print wrong parameters and go on
    # AIC criteria: 
    except:
        print('wrong parameters:', param)
        continue
    aic = model.aic

    # save best model, aic, parameters
    if aic < best_aic:
        best_model = model
        best_aic = aic
        best_param = param
        results.append([param, model.aic])
    
warnings.filterwarnings('default')

In [ ]:
result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'aic']
print(result_table.sort_values(by = 'aic', ascending=True).head())

In [ ]:
print(best_model.summary())

In [ ]:
#смотрим на остатки: они д.б. нормально распределены (p-value не д.б. <0.05) и стационарны (p-value<0.05 для ДФ теста)
plt.subplot(211)
best_model.resid[d:].plot()
plt.ylabel('Residuals')

ax = plt.subplot(212)
sm.graphics.tsa.plot_acf(best_model.resid[d:].values.squeeze(), lags=23, ax=ax)

print("Student test: p=%f" % stats.ttest_1samp(best_model.resid[d:], 0)[1])
print("Dickey-Fuller test: p=%.25f" % sm.tsa.stattools.adfuller(best_model.resid[d:])[1])

In [ ]:
history = [x for x in df_train['Volume of mortgage loans'].values]
predictions = list()

In [ ]:
for t in range(len(df_test)):
    model = sm.tsa.statespace.SARIMAX(history, order=(p_opt,d,q_opt), seasonal_order=(0,0,0,0))
    model_fit = model.fit()
    output = model_fit.predict(start=len(df_train)+t, end=len(df_train)+t+1)
    yhat = output[0]
    predictions.append(yhat)
    obs = df_test['Volume of mortgage loans'].values[t]
    history.append(obs)

In [ ]:
def MAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
MAPE(df_test['Volume of mortgage loans'], predictions)

In [ ]:
#ARIMAX
history = [x for x in df_train['Volume of mortgage loans'].values]
ex1 = [x for x in df_train['Russian Central Bank Key rate'].values]
ex2 = [x for x in df_train['Dollar/ruble exchange rate'].values]
ex3 = [x for x in df_train['Yandex Query 1'].values]
ex = np.transpose(np.array([ex1, ex2, ex3]))
predictions = list()

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
for t in range(len(df_test)):
    model = SARIMAX(history, exog=ex, order=(p_opt,d,q_opt), seasonal_order=(0,0,0,0))
    model_fit = model.fit()
    exog1, exog2, exog3 = [], [], []
    exog1.append(df_test['Russian Central Bank Key rate'].values[t])
    exog2.append(df_test['Dollar/ruble exchange rate'].values[t])
    exog3.append(df_test['Yandex Query 1'].values[t])
    exog = np.transpose(np.array([exog1, exog2, exog3]))
    output = model_fit.predict(start=len(df_train)+t, end=len(df_train)+t, exog=exog)
    predictions.append(output[0])
    obs = df_test['Volume of mortgage loans'].values[t]
    history.append(obs)
    ex = np.vstack((ex, exog))

In [ ]:
MAPE(df_test['Volume of mortgage loans'], predictions)